## Task 1 - *Exploration of customer purchasing behavior*

### *Importing liberaries*

In [1]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import os,sys
sys.path.append(os.path.abspath(os.path.join('..')))
from scripts.eda import EDA

ModuleNotFoundError: No module named 'scripts.eda'